In [1]:
import os
import tensorflow as tf

In [2]:
print(tf.__version__)

2.5.0


In [3]:
SAVED_MODEL_PATH = './conv_2d.tflite'

In [4]:
interpreter = tf.lite.Interpreter(model_path=SAVED_MODEL_PATH)
interpreter

In [5]:
help(interpreter)

Help on Interpreter in module tensorflow.lite.python.interpreter object:

class Interpreter(builtins.object)
 |  Interpreter interface for TensorFlow Lite Models.
 |  
 |  This makes the TensorFlow Lite interpreter accessible in Python.
 |  It is possible to use this interpreter in a multithreaded Python environment,
 |  but you must be sure to call functions of a particular instance from only
 |  one thread at a time. So if you want to have 4 threads running different
 |  inferences simultaneously, create  an interpreter for each one as thread-local
 |  data. Similarly, if you are calling invoke() in one thread on a single
 |  interpreter but you want to use tensor() on another thread once it is done,
 |  you must use a synchronization primitive between the threads to ensure invoke
 |  has returned before calling tensor().
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |  
 |  __init__(self, model_path=None, model_content=None, experimental_delegates=None, num_threads=None, ex

In [26]:
help(interpreter._interpreter)

Help on InterpreterWrapper in module tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper object:

class InterpreterWrapper(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      InterpreterWrapper
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  AllocateTensors(...)
 |      AllocateTensors(self: tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper.InterpreterWrapper) -> object
 |  
 |  GetOutputTensorFromSignatureDefName(...)
 |      GetOutputTensorFromSignatureDefName(self: tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper.InterpreterWrapper, arg0: str, arg1: str) -> object
 |  
 |  GetSignatureDefs(...)
 |      GetSignatureDefs(self: tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper.InterpreterWrapper) -> object
 |  
 |  GetTensor(...)
 |      GetTensor(self: tensorflow.lite.python.interpre

In [6]:
interpreter.get_tensor_details()

[{'name': 'MobilenetV1/Conv2d_0/weights',
  'index': 0,
  'shape': array([32,  3,  3,  3], dtype=int32),
  'shape_signature': array([32,  3,  3,  3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'MobilenetV1/MobilenetV1/Conv2d_0/Relu6',
  'index': 1,
  'shape': array([  1, 112, 112,  32], dtype=int32),
  'shape_signature': array([  1, 112, 112,  32], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'MobilenetV1/MobilenetV1/Conv2d_0/convolution_bias',
  'index': 2,
  'shape': array([32], dtype=int32),
  'shape_signature': array([32], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0,

In [7]:
interpreter._get_ops_details()

[{'index': 0,
  'op_name': 'CONV_2D',
  'inputs': array([3, 0, 2], dtype=int32),
  'outputs': array([1], dtype=int32)}]

```c
  const tflite::Interpreter* interpreter = ...; // Initialize interpreter
  for (int op_index : interpreter->execution_plan()) {
    const auto* op_and_reg = interpreter->node_and_registration(op_index);
    if (op_and_reg->second.builtin_code == kTfLiteBuiltinConv2d) {
      // Parse operation basing on its type.
      // See other types in tensorflow/lite/builtin_ops.h
    }
    for (const auto input_idx :
         tflite::TfLiteIntArrayView(op_and_reg->first.inputs)) {
      // Access operation input using its index.
    }
    for (const auto output_idx :
         tflite::TfLiteIntArrayView(op_and_reg->first.outputs)) {
      // Access operation output using its index.
    }
  }
```

In [8]:
interpreter._interpreter.NumNodes()

1

In [9]:
interpreter2 = tf.lite.Interpreter(model_path='./MODELS/densenet.tflite')

In [10]:
# todo : 이게 왜 inputs와 outputs가 다른건지 한번 물어보자. 이런 쉐입은 그동안 없었는데 말이다..
for op_index in range(interpreter2._interpreter.NumNodes()):
    op_and_reg = interpreter2._get_op_details(op_index)
    print(op_and_reg)
    if op_index == 10:
        break

{'index': 0, 'op_name': 'CONV_2D', 'inputs': array([  0, 491, 490], dtype=int32), 'outputs': array([1], dtype=int32)}
{'index': 1, 'op_name': 'MAX_POOL_2D', 'inputs': array([1], dtype=int32), 'outputs': array([497], dtype=int32)}
{'index': 2, 'op_name': 'MUL', 'inputs': array([497,  13], dtype=int32), 'outputs': array([12], dtype=int32)}
{'index': 3, 'op_name': 'ADD', 'inputs': array([12, 11], dtype=int32), 'outputs': array([10], dtype=int32)}
{'index': 4, 'op_name': 'CONV_2D', 'inputs': array([10, 17, 16], dtype=int32), 'outputs': array([15], dtype=int32)}
{'index': 5, 'op_name': 'CONCATENATION', 'inputs': array([497,  15], dtype=int32), 'outputs': array([14], dtype=int32)}
{'index': 6, 'op_name': 'MUL', 'inputs': array([14, 21], dtype=int32), 'outputs': array([20], dtype=int32)}
{'index': 7, 'op_name': 'ADD', 'inputs': array([20, 19], dtype=int32), 'outputs': array([18], dtype=int32)}
{'index': 8, 'op_name': 'CONV_2D', 'inputs': array([18, 25, 24], dtype=int32), 'outputs': array([23]

In [11]:
interpreter2._get_ops_details()[:10]

[{'index': 0,
  'op_name': 'CONV_2D',
  'inputs': array([  0, 491, 490], dtype=int32),
  'outputs': array([1], dtype=int32)},
 {'index': 1,
  'op_name': 'MAX_POOL_2D',
  'inputs': array([1], dtype=int32),
  'outputs': array([497], dtype=int32)},
 {'index': 2,
  'op_name': 'MUL',
  'inputs': array([497,  13], dtype=int32),
  'outputs': array([12], dtype=int32)},
 {'index': 3,
  'op_name': 'ADD',
  'inputs': array([12, 11], dtype=int32),
  'outputs': array([10], dtype=int32)},
 {'index': 4,
  'op_name': 'CONV_2D',
  'inputs': array([10, 17, 16], dtype=int32),
  'outputs': array([15], dtype=int32)},
 {'index': 5,
  'op_name': 'CONCATENATION',
  'inputs': array([497,  15], dtype=int32),
  'outputs': array([14], dtype=int32)},
 {'index': 6,
  'op_name': 'MUL',
  'inputs': array([14, 21], dtype=int32),
  'outputs': array([20], dtype=int32)},
 {'index': 7,
  'op_name': 'ADD',
  'inputs': array([20, 19], dtype=int32),
  'outputs': array([18], dtype=int32)},
 {'index': 8,
  'op_name': 'CONV_2D'

In [12]:
print(len(interpreter2._get_ops_details()))

249


In [13]:
interpreter2.get_tensor_details()[5]

{'name': 'block-0/batch_normalization/FusedBatchNorm_mul_0',
 'index': 5,
 'shape': array([  1,  56,  56, 256], dtype=int32),
 'shape_signature': array([  1,  56,  56, 256], dtype=int32),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [14]:
help(interpreter2._interpreter)

Help on InterpreterWrapper in module tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper object:

class InterpreterWrapper(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      InterpreterWrapper
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  AllocateTensors(...)
 |      AllocateTensors(self: tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper.InterpreterWrapper) -> object
 |  
 |  GetOutputTensorFromSignatureDefName(...)
 |      GetOutputTensorFromSignatureDefName(self: tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper.InterpreterWrapper, arg0: str, arg1: str) -> object
 |  
 |  GetSignatureDefs(...)
 |      GetSignatureDefs(self: tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper.InterpreterWrapper) -> object
 |  
 |  GetTensor(...)
 |      GetTensor(self: tensorflow.lite.python.interpre

In [15]:
os.path.dirname('__file__')

''

In [16]:
interpreter2._get_ops_details()[:2]

[{'index': 0,
  'op_name': 'CONV_2D',
  'inputs': array([  0, 491, 490], dtype=int32),
  'outputs': array([1], dtype=int32)},
 {'index': 1,
  'op_name': 'MAX_POOL_2D',
  'inputs': array([1], dtype=int32),
  'outputs': array([497], dtype=int32)}]

In [17]:
interpreter2.get_tensor_details()[:2]

[{'name': 'Placeholder',
  'index': 0,
  'shape': array([  1, 224, 224,   3], dtype=int32),
  'shape_signature': array([  1, 224, 224,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'Relu',
  'index': 1,
  'shape': array([  1, 112, 112,  64], dtype=int32),
  'shape_signature': array([  1, 112, 112,  64], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [18]:
ops = interpreter2._get_ops_details()
cnt = 0
for op_index, op in enumerate(ops):
    if op['op_name'] == "CONV_2D":
        cnt += 1
        for tensor_idx in op['inputs']:
            tensor = interpreter2._get_tensor_details(tensor_idx)
            tensor_shape = tensor['shape']
            #print(tensor_shape)
            print(tensor['name'], "\t", tensor['shape'])
        print("----")

Placeholder 	 [  1 224 224   3]
conv2d/kernel 	 [64  7  7  3]
conv2d/Conv2D_bias 	 [64]
----
block-0/denseblock-0-0/Relu 	 [ 1 56 56 64]
block-0/denseblock-0-0/conv2d/kernel 	 [32  3  3 64]
block-0/denseblock-0-0/conv2d/Conv2D_bias 	 [32]
----
block-0/denseblock-0-1/Relu 	 [ 1 56 56 96]
block-0/denseblock-0-1/conv2d/kernel 	 [32  3  3 96]
block-0/denseblock-0-1/conv2d/Conv2D_bias 	 [32]
----
block-0/denseblock-0-2/Relu 	 [  1  56  56 128]
block-0/denseblock-0-2/conv2d/kernel 	 [ 32   3   3 128]
block-0/denseblock-0-2/conv2d/Conv2D_bias 	 [32]
----
block-0/denseblock-0-3/Relu 	 [  1  56  56 160]
block-0/denseblock-0-3/conv2d/kernel 	 [ 32   3   3 160]
block-0/denseblock-0-3/conv2d/Conv2D_bias 	 [32]
----
block-0/denseblock-0-4/Relu 	 [  1  56  56 192]
block-0/denseblock-0-4/conv2d/kernel 	 [ 32   3   3 192]
block-0/denseblock-0-4/conv2d/Conv2D_bias 	 [32]
----
block-0/denseblock-0-5/Relu 	 [  1  56  56 224]
block-0/denseblock-0-5/conv2d/kernel 	 [ 32   3   3 224]
block-0/denseblock-0-5/

In [20]:
ops = interpreter2._get_ops_details()
cnt = 0
for op_index, op in enumerate(ops):
    if op['op_name'] == "CONV_2D":
        cnt += 1
        for tensor_idx in op['inputs']:
            tensor = interpreter2._get_tensor_details(tensor_idx)
            tensor_shape = tensor['shape']
            #print(tensor_shape)
            print(tensor['name'], "\t", tensor['shape'])
            print(tensor)
        print("----")
    if op_index == 10:
        break

Placeholder 	 [  1 224 224   3]
{'name': 'Placeholder', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
conv2d/kernel 	 [64  7  7  3]
{'name': 'conv2d/kernel', 'index': 491, 'shape': array([64,  7,  7,  3], dtype=int32), 'shape_signature': array([64,  7,  7,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
conv2d/Conv2D_bias 	 [64]
{'name': 'conv2d/Conv2D_bias', 'index': 490, 'shape': array([64], dtype=int32), 'shape_signature': array([64], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), '

In [21]:
analyzer = tf.lite.ModelAnalyzer(interpreter2, "txt")
analyzer

AttributeError: module 'tensorflow._api.v2.lite' has no attribute 'ModelAnalyzer'

In [28]:
type(interpreter._interpreter)


tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper.InterpreterWrapper